## KNN com Surprise

In [ ]:
!pip install scikit-surprise -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader

A biblioteca Surprise tem uma classe chamada KNNWithMeans que pode ser usada para criar um recomendador baseado em filtragem colaborativa usando o algoritmo k-NN. Aqui está um exemplo de como criar e treinar um recomendador usando essa classe:

In [ ]:
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
from surprise import KNNWithMeans

In [ ]:
# Usando item-based com similaridade cosseno
sim_options_1 ={'name': 'pearson_baseline', 'user_based': True}
sim_options_2 = {"name": "cosine", "user_based": False,}  # Computar similaridade entre os itens

In [ ]:
# Define o algoritmo a ser usado
algo = KNNWithMeans(k=50, sim_options=sim_options_2)

A função de recomendação acima é configurada para usar a similaridade de cosseno e encontrar itens semelhantes usando a abordagem baseada em itens.

Iremos criar um dataset de treino a partir dos dados. 

In [ ]:
# Treina o algoritmo
trainset = data.build_full_trainset()

In [ ]:
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Faz previsões
uid = str(196)  # usuário id
iid = str(302)  # item id
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.18   {'actual_k': 39, 'was_impossible': False}


# Ajustando os Parâmetros do Algoritmo
Surprise fornece uma classe GridSearchCV análoga a GridSearchCV do scikit-learn.

Com um dict de todos os parâmetros, GridSearchCV tenta todas as combinações de parâmetros e relata os melhores parâmetros para qualquer medida de precisão

Por exemplo, você pode verificar qual métrica de similaridade funciona melhor para seus dados em abordagens baseadas em memória:

- name contém a métrica de similaridade a ser usada. As opções são cosseno, msd, pearson ou pearson_baseline. O padrão é msd.

- user_based booleano que informa se a abordagem será baseada no usuário ou no item. O padrão é True, o que significa que a abordagem baseada no usuário será usada.

- min_support é o número mínimo de itens comuns necessários entre os usuários para considerá-los por similaridade. Para a abordagem baseada em itens, isso corresponde ao número mínimo de usuários comuns para dois itens.

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["pearson_baseline", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

In [ ]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])